# The Bayesian Workflow OR A Taste of Probabilistic Programming

## Bayesian parameter estimation using PyMC

Now it's time to generalize our new skills and build on them by learning the basics of probabilistic programming using PyMC. To this end, let's recap the general workflow. 

In the basics of Bayesian model building, the steps are
1. To completely specify the model in terms of _probability distributions_. This includes specifying 
    - what the form of the sampling distribution of the data is _and_ 
    - what form describes our _uncertainty_ in the unknown parameters.
2. Calculate the _posterior distribution_.

In Chapter 5 and 6's biased coin flipping example, the form of the sampling distribution of the data was Binomial (described by the likelihood) and the uncertainty around the unknown parameter $p$ captured by the prior (we used both the uniform and the Jeffreys priors).

Now it is time to do the same using the **probabilistic programming language** PyMC. There's _loads of cool stuff_ about PyMC and this paradigm, two of which are
- _probabililty distributions_ are first class citizens, in that we can assign them to variables and use them intuitively to mirror how we think about priors, likelihoods & posteriors.
- PyMC calculates the posterior for us!

Under the hood, PyMC will compute the posterior using a sampling based approach called Markov Chain Monte Carlo (MCMC) or Variational Inference. Check the [PyMC3 docs](https://docs.pymc.io/) for more on these. 

But now, it's time to bust out some MCMC and get sampling!

### Parameter estimation I: click-through rate

A common experiment in tech data science is to test a product change and see how it affects a metric that you're interested in. Say that I don't think enough people are clicking a button on my website & I hypothesize that it's because the button is a similar color to the background of the page. Then I can set up two pages and send some people to each: the first the original page, the second a page that is identical, except that it has a button that is of higher contrast and see if more people click through. This is commonly referred to as an A/B test and the metric of interest is click-through rate (CTR), what proportion of people click through. Before even looking at two rates, let's use PyMC to estimate one.



First lets generate click-through data, given a CTR $p_a=0.15$.

In [ ]:
#| output: false
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az
from IPython.display import display_png
from generative_thinking.utils import ecdf 

%matplotlib inline
sns.set()

In [ ]:
# click-through rates
np.random.seed(42)
p_a = 0.15
N = 150
n_successes_a = np.sum(np.random.binomial(N, p_a))

Now it's time to build our probability model. Noticing that our model of having a constant CTR resulting in click or not is a biased coin flip,
- the sampling distribution is binomial and we need to encode this in the likelihood;
- there is a single parameter $p$ that we need to describe the uncertainty around, using a prior and we'll use a uniform prior for this.

These are the ingredients for the model so let's now build it:

In [ ]:
# Build model of p_a
with pm.Model() as model:
    # Prior on p
    prob = pm.Uniform('p')
    # Binomial Likelihood
    y = pm.Binomial('y', n=N, p=prob, observed=n_successes_a)


Now that we've done the hard work of building our model, 
let's visualize the model graph using `graphviz`.
In our case, the graph will look simple
because our model doesn't have too many moving parts.
That said, being able to see the model graph is awesome
and will introduce you with a tool
that will help visualizing more complex models in the future.

In [ ]:
gv = pm.model_graph.model_to_graphviz(model)
display_png(gv)

It's now time to sample from the posterior using PyMC. You'll also then plot the posterior:

In [ ]:
#| output: false
with model:
    samples = pm.sample(2000, return_inferencedata=True)
    # We're also sampling the prior/posterior predictives,
    # which we'll introduce and explain below
    idata = pm.sample_prior_predictive(model=model)
    idata.extend(pm.sample(progressbar=False))
    idata.extend(pm.sample_posterior_predictive(idata, progressbar=False))


In [ ]:
#| fig-cap: Posterior samples of $p$, the click-through rate, having seen the data.
with model:
    az.plot_posterior(samples, kind='hist');

#### Predictive checks and traceplots

Let's now introduce 3 diagnostic tools, which will come in handy when building models:

- prior predictive check,
- posterior predictive check, and
- traceplots.

**Prior predictive checks** essentially generate samples of the data
based on your specified priors without looking at the actual data.
They help you check whether your priors are sensible distributions,
or if they have the potential to generate wildly unreasonable data.
Let's plot the prior predictive check ECDF for our model above:

In [ ]:
#| fig-cap: Prior predictive samples for observed data.
# Generate x & y data for ECDF
x_ecdf, y_ecdf = ecdf(idata.prior_predictive["y"].data.flatten())
# Plot the ECDF
plt.plot(x_ecdf, y_ecdf, marker='.', linestyle='none');
plt.xlabel("Number of Clicks")
plt.ylabel("Cumulative Fraction");

Now, we have to ask the question: Does this prior predictive check look sensible?
This question is both qualitative and quantitative.
Qualitatively, it looks extremely similar to a uniform distribution's CDF.
Quantitatively, its lower bound $0$ and upper bound $150$ are what we expect.
We know that we are sampling the number of click throughs
out of a total number of 150 website landings
so this prior distribution looks pretty good!

Counterfactually, we would be concerned if there negative values _or_ values over $150$,
as they would violate what we assume about our data-generating processes.
In the next example, we'll see how red flags in our prior predictive check
can lead us to alter our priors.

The *posterior* predictive check samples our posterior distribution of data so we would expect it to look something like our actual data. Let's check it out, in this example:

In [ ]:
# Generate x & y data for ECDF
x_ecdf, y_ecdf = ecdf(idata.posterior_predictive["y"].data.flatten())
# Plot the ECDF
plt.plot(x_ecdf, y_ecdf, marker='.', linestyle='none', color="black")
plt.axvline(x=n_successes_a, color="black");

Notice that this looks somewhat Gaussian, centered around 21. Now our data is a single number, the number of clicks, so we would expect it to be around $21\pm5$. Let's see what it is:

In [ ]:
n_successes_a

This looks good.

Finally, let's now plot the *traceplot*, which plots our parameters as a function of iteration number and tells us about whether our sampling has converged. Essentially we want to make sure that the sampling procedure doesn't "shoot off" in any weird way, such as in the following:

In [ ]:
#| echo: false
from IPython.display import Image
Image("../../img/badchains.png")

What we really want to see is a traceplot that doesn't exhibit drift but diffusion,
that is, noise around a certain value.
There are ways to frame this in terms of autocorrelation but,
for the time being, the most important way to think about it is
you want to see a traceplot that looks something like a "hairy caterpillar",
which we do on the right in this case
(on the left we have the samples of our distribution):

In [ ]:
az.plot_trace(samples);

### Parameter estimation II -- the mean of a population

We'll now calculate the  posterior mean beak depth of Galapagos finches in a given species. First let's load the data and subset it with respect to species:

In [ ]:
# Import and view head of data
df_12 = pd.read_csv('../../datasets/finch_beaks_2012.csv')
df_fortis = df_12.loc[df_12['species'] == 'fortis']
df_scandens = df_12.loc[df_12['species'] == 'scandens']

To specify the full probability model, we need
- a likelihood function for the data &
- priors for all unknowns.

What is the likelihood here? Let's plot the measurements below and see that they look approximately normal so we'll use a normal likelihood $y_i\sim \mathcal{N}(\mu, \sigma^2)$. The unknowns here are the mean $\mu$ and standard deviation $\sigma$ and we'll use weakly informative priors on both

- a normal prior for $\mu$ with mean $10$ and standard deviation $5$;
- a lognormal prior for $\sigma$ with $\mu = 0$ and $\sigma = 10$.

There are biological reasons for these priors 
(**NOTE: not really now, with the lognormal prior? 
Need to explain what the biological reasons are otherwise**)
also but we can also test that the posteriors are relatively robust to the choice of prior here due to the amount of data.

In [ ]:
#| warnings:false
sns.distplot(df_fortis['blength']);

Let's now build our model:

In [ ]:
with pm.Model() as model:
    # Prior for mean & standard deviation
    mu_1 = pm.Normal('mu_1', mu=10, sigma=5)
    sigma_1 = pm.Lognormal('sigma_1', 0, 10)
    # Gaussian Likelihood
    y_1 = pm.Normal('y_1', mu=mu_1, sigma=sigma_1,
     observed=df_fortis['blength'])

Once again, we can also visualize our model graph:

In [ ]:
gv = pm.model_graph.model_to_graphviz(model)
display_png(gv)

We now press our inference button and sample from the posterior:

In [ ]:
#| output: false
with model:
    samples = pm.sample(2000, return_inferencedata=True)
    idata = pm.sample_prior_predictive(model=model)
    idata.extend(pm.sample(progressbar=False))
    idata.extend(pm.sample_posterior_predictive(idata, progressbar=False))


Let's now plot our *prior predictive check*, data, and *posterior predictive check*:

In [ ]:
x_ecdf, y_ecdf = ecdf(idata.prior_predictive["y_1"].data.flatten())
# Plot the ECDF
plt.plot(x_ecdf, y_ecdf, marker='.', linestyle='none');

In [ ]:
x_ecdf, y_ecdf = ecdf(df_fortis['blength'].values)
# Plot the ECDF
plt.plot(x_ecdf, y_ecdf, marker='.', linestyle='none');

In [ ]:
x_ecdf, y_ecdf = ecdf(idata.posterior_predictive["y_1"].data.flatten())
# Plot the ECDF
plt.plot(x_ecdf, y_ecdf, marker='.', linestyle='none');

Although the *posterior predictive check* passes the eyeball test (when comapring with the data), the *prior predictive check* does not! Looking at the x-axis on it, we see that we end up with values orders of magnitude larger than what a beak length could possibly be!

This means that our priors aren't quite right yet.
Looking at them, we can see that having a Lognormal prior for our variance
means that we could end up with some pretty large values
so let's try a prior that drops off more quickly (we say it has a _shorter tail_).
We'll try an Exponential prior here as we know that these have shorter tails.
We don't expect the reader to be aware of this
but we hope to help build your intuition around this here
and in the following case stories!

In [ ]:
with pm.Model() as model:
    # Prior for mean & standard deviation
    mu_1 = pm.Normal('mu_1', mu=10, sigma=5)
    sigma_1 = pm.Exponential('sigma_1', lam=0.5)
    # Gaussian Likelihood
    y_1 = pm.Normal('y_1', mu=mu_1, sigma=sigma_1, 
    observed=df_fortis['blength'])

In [ ]:
#| output: false
with model:
    samples = pm.sample(2000, return_inferencedata=True)
    idata = pm.sample_prior_predictive(model=model)
    idata.extend(pm.sample(progressbar=False))
    idata.extend(pm.sample_posterior_predictive(idata, 
        progressbar=False))


Let's now plot our *prior* and *posterior* predictive checks:

In [ ]:
x_ecdf, y_ecdf = ecdf(idata.prior_predictive["y_1"].data.flatten())
# Plot the ECDF
plt.plot(x_ecdf, y_ecdf, marker='.', linestyle='none');

In [ ]:
x_ecdf, y_ecdf = ecdf(idata.posterior_predictive["y_1"].data.flatten())
# Plot the ECDF
plt.plot(x_ecdf, y_ecdf, marker='.', linestyle='none');

The *prior* predictive check is now looking a lot better!
There is still the possibility of negative values, so that may still be an issue.
Additionally, if it were still generating data that are too large,
we could increase the exponent `lam`, among other things.

Now let's plot the posteriors of our two parameters, mean $\mu$ and standard deviation $\sigma$:

In [ ]:
with model:
    az.plot_posterior(samples);

We once again plot the traceplot, which plots our parameters as a function of iteration number, showing that our model has converged:

In [ ]:
az.plot_trace(samples, figsize=(10, 7));

## Bayesian hypothesis testing

After parameter estimation, a common scientific question involves hypothesis testing, so let's now see how Bayesian inference and probabilistic programming can be used for hypothesis testing.

### Bayesian hypothesis testing I: A/B tests on click through rates

Assume we have a website and want to redesign the layout (*A*) and test whether the new layout (*B*) results in a higher click through rate. When people come to our website we randomly show them layout *A* or *B* and see how many people click through for each. First let's generate the data we need:

In [ ]:
# click-through rates
p_a = 0.15
p_b = 0.20
N = 1000
n_successes_a = np.sum(np.random.uniform(size=N) <= p_a)
n_successes_b = np.sum(np.random.uniform(size=N) <= p_b)

Once again, we need to specify our models for $p_a$ and $p_b$. Each will be the same as the CTR example above
- Binomial likelihoods
- uniform priors on $p_a$ and $_p$.

We also want to calculate the posterior of the difference $p_a-p_b$ and we do so using `pm.Deterministic()`, which specifies a deterministic random variable, i.e., one that is completely determined by the values it references, in the case $p_a$ & $p_b$.

We'll now build the model:

In [ ]:
with pm.Model() as model:
    # Prior on p
    prob_a = pm.Uniform('p_a')
    prob_b = pm.Uniform('p_b')
    # Binomial Likelihood
    y_a = pm.Binomial('y_a', n=N, p=prob_a, observed=n_successes_a)
    y_b = pm.Binomial('y_b', n=N, p=prob_b, observed=n_successes_b)
    diff_clicks = pm.Deterministic('diff_clicks', prob_a-prob_b)

Let's now plot our model graph:

In [ ]:
gv = pm.model_graph.model_to_graphviz(model)
display_png(gv)

Sample from the posterior and plot them:

In [ ]:
#| output: false
with model:
    samples = pm.sample(2000, return_inferencedata=True)
    az.plot_posterior(samples, kind='hist');

We once again plot the traceplot, which plots our parameters as a function of iteration number, showing that our model has converged:

In [ ]:
az.plot_trace(samples, figsize=(10, 7));

Note that, for the sake of brevity, we did not perform our predictive checks, but this is something that the excited reader can do, in this case, and for those below!

### Bayesian hypothesis testing II -- beak lengths difference between species

In the following, we seek to determine whether the mean beak length of the Galapagos finches differs between species. For the mean of each species, we use the same model as in previous section:

- Gaussian likelihood;
- Normal prior for the means;
- Uniform prior for the variances.

We also calculate the difference between the means and, the _effect size_, which is the difference between the means divided by the pooled standard deviations = $\sqrt{(\sigma_1^2+\sigma_2^2)/2}$. 

We then sample from the posteriors and plot them!

In [ ]:
with pm.Model() as model:
    # Priors for means and variances
    mu_1 = pm.Normal('mu_1', mu=10, sigma=5)
    sigma_1 = pm.Uniform('sigma_1', 0, 10)
    mu_2 = pm.Normal('mu_2', mu=10, sigma=5)
    sigma_2 = pm.Uniform('sigma_2', 0, 10)
    # Gaussian Likelihoods
    y_1 = pm.Normal('y_1', mu=mu_1, sigma=sigma_1, observed=df_fortis['blength'])
    y_2 = pm.Normal('y_2', mu=mu_2, sigma=sigma_2, observed=df_scandens['blength'])
    # Calculate the effect size and its uncertainty.
    diff_means = pm.Deterministic('diff_means', mu_1 - mu_2)
    pooled_sd = pm.Deterministic('pooled_sd', 
                                 np.sqrt(np.power(sigma_1, 2) + 
                                         np.power(sigma_2, 2)) / 2)
    effect_size = pm.Deterministic('effect_size', 
                                   diff_means / pooled_sd)

Let's now plot our model graph:

In [ ]:
gv = pm.model_graph.model_to_graphviz(model)
display_png(gv)

In [ ]:
#| output: false
with model:
    samples = pm.sample(2000, return_inferencedata=True)

In [ ]:
az.plot_posterior(samples, 
    var_names=['mu_1', 'mu_2', 'diff_means', 'effect_size'], kind='hist');

In [ ]:
az.plot_trace(samples, figsize=(10, 7));

## Bayesian regression

In the above, we have seen how to use the Bayesian workflow and probabilistic programming for both parameter estimation and comparison between groups. To now round out our first few Bayesian modeling tools, let's see how to use the Bayesian workflow and probabilistic programming for linear regression!

To this end, let's first generate some bivariate data, consisting of a linear relationship and a noise/error term:

In [ ]:
np.random.seed(42)
(m, b) = (25, 0.5)
x = 100 * np.random.random(20)
y = m * x + b

# add scatter to points
x = np.random.normal(x, 10)
y = np.random.normal(y, 10)

plt.plot(x, y, 'ok');


Now recall that our Bayesian model building steps are
1. To completely specify the model in terms of _probability distributions_. This includes specifying 
    - what the form of the sampling distribution of the data is _and_ 
    - what form describes our _uncertainty_ in the unknown parameters.
2. Calculate the _posterior distribution_.

The sampling distribution of the data is given by $y = mx + b + \sigma$, where $m$ is the gradient, $b$ the $y$-intercept, and $\sigma$ the error/noise term. These are the unknown parameters, which we need to describe our _uncertatinty_ around using priors and we choose:

- Gaussian priors for $m$ and $b$,
- A Half-Cauchy prior for $\sigma$ (this is a distribution that has desirable properties for us, but isn't worth getting into the details of)

In [ ]:
with pm.Model() as model:
    # Define priors
    sigma = pm.HalfCauchy("sigma", beta=10)
    b = pm.Normal("b", 0, sigma=20)
    m = pm.Normal("m", 0, sigma=20)

    # Define likelihood
    likelihood = pm.Normal("y", mu=b + m * x, sigma=sigma, observed=y)

Let's now plot our model graph:

In [ ]:
gv = pm.model_graph.model_to_graphviz(model)
display_png(gv)

In [ ]:
#| output: false
with model:
    samples = pm.sample(2000, return_inferencedata=True)


We'll see in following chapter how to plot and interpret predictive checks for regression tasks, such as this.

Let's now plot the traceplot, which plots our parameters as a function of iteration number and here shows that our model has converged in a desirable way:

In [ ]:
az.plot_trace(samples, figsize=(10, 7));

## Wrapping up

In this chapter, we've introduced the fundamentals of the principled Bayesian workflow and demonstrated its utility in answering the common scientific questions of parameter estimation and hypothesis testing. We've done so for two different types of data, the first an example of data from A/B testing, the second a dataset from basic research of Galapagos finch beak measurements. We also introduced Bayesian linear regression.